In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
#internal
import os, sys, glob, yaml, datetime, argparse
import csv
import tensorflow as tf

Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])

class GraphDataset():
    def __init__(self, input_dir, n_samples=None):
        input_dir = os.path.expandvars(input_dir)
        filenames = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.startswith('event') and f.endswith('.npz')]
        self.filenames = (
            filenames[:n_samples] if n_samples is not None else filenames)

    def __getitem__(self, index):
        return load_graph(self.filenames[index])

    def __len__(self):
        return len(self.filenames)
def get_dataset(input_dir,n_files):
    return GraphDataset(input_dir, n_files)
def load_graph(filename):
    """Reade a single graph NPZ"""
    with np.load(filename) as f:
        return sparse_to_graph(**dict(f.items()))
def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.float32):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)

In [3]:
current_directory = os.getcwd()

In [4]:
get_dataset(current_directory,1)

In [9]:
graph_ex=load_graph('event000001000_g000.npz')

In [19]:
X=graph_ex.X
Ri=graph_ex.Ri
Ro=graph_ex.Ro
y=graph_ex.y

In [13]:
X.shape

(6437, 3)

In [16]:
print("X shape: ", X.shape," Ri shape: ", Ri.shape,
      " Ro shape: ", Ro.shape," y shape: ", y.shape)

X shape:  (6437, 3)  Ri shape:  (6437, 10670)  Ro shape:  (6437, 10670)  y shape:  (10670,)


In [26]:
y=y.astype(np.int32)
Ri = Ri.astype(np.int32)
Ro = Ro.astype(np.int32)

In [ ]:
#Create coloring label
def graph_coloring(X, Ri, Ro, y):
    NV, NE = Ri.shape
    visited = np.zeros(NV, dtype=bool)
    colors = np.zeros(NV, dtype=int)
    
    def dfs(node, color):
        visited[node] = True
        colors[node] = color
        
        for edge in range(NE):
            if Ri[node, edge] == 1 and not visited[Ro[node, edge]]:
                dfs(Ro[node, edge], color)
    
    color = 0
    for node in range(NV):
        if not visited[node]:
            dfs(node, color)
            color += 1
    
    return colors

colors = graph_coloring(X, Ri, Ro, y)
print("Graph Coloring:", colors)